In [ ]:
# this was for testing the model


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from flask import Flask, jsonify, request

file_path = 'employee_performance_cleaned_v2.xlsx'
df = pd.read_excel(file_path)

print(df.shape)
df.head()


(1484, 10)


,Employee Name,Email_ID,Team,SubTeam,Task,Skillset,Efficiency,Feedback,SprintNo.,Employee_ID
0,Yuvraj,yuvraj2.mitblr2023@learner.manipal.edu,L1,Spec-1,Documentation,"Confluence, Markdown, Technical Writing",90,Very efficient,Sprint 1,1020
1,Yuvraj,yuvraj2.mitblr2023@learner.manipal.edu,L1,Spec-1,Backend API,"Express, Flask, Node.js",89,Clear and structured,Sprint 2,1020
2,Yuvraj,yuvraj2.mitblr2023@learner.manipal.edu,L1,Spec-1,Database Optimization,"MongoDB, SQL, Indexing",95,Very efficient,Sprint 3,1020
3,Yuvraj,yuvraj2.mitblr2023@learner.manipal.edu,L1,Spec-1,Database Optimization,"SQL, MongoDB, PostgreSQL",80,Very efficient,Sprint 5,1020
4,Yuvraj,yuvraj2.mitblr2023@learner.manipal.edu,L1,Spec-1,Testing,"Cypress, Manual Testing, PyTest",61,"Needs improvement, delayed delivery",Sprint 6,1020


In [ ]:
unique_feedbacks = df['Feedback'].dropna().unique()
print("Unique feedback responses found:\n", unique_feedbacks)

def score_feedback(feedback):
    if pd.isna(feedback):
        return 3
    f = str(feedback).lower()
    if "excellent" in f or "very efficient" in f:
        return 5
    elif "good" in f or "clear" in f or "creative" in f:
        return 4
    elif "average" in f:
        return 3
    elif "needs improvement" in f or "issues" in f or "delayed" in f:
        return 2
    else:
        return 3  

df['feedback_score'] = df['Feedback'].apply(score_feedback)

def normalize_skillset(text):
    if pd.isna(text): 
        return []
    return [p.strip().lower() for p in str(text).split(",")]

df['skill_list'] = df['Skillset'].apply(normalize_skillset)

df[['Feedback', 'feedback_score', 'Skillset', 'skill_list']].head()


Unique feedback responses found:
 ['Very efficient' 'Clear and structured'
 'Needs improvement, delayed delivery' 'Excellent, thorough coverage'
 'Good but slightly delayed' 'Creative and user-friendly'
 'Average execution' 'Code quality issues' 'Creative' 'Needs improvement']


,Feedback,feedback_score,Skillset,skill_list
0,Very efficient,5,"Confluence, Markdown, Technical Writing","[confluence, markdown, technical writing]"
1,Clear and structured,4,"Express, Flask, Node.js","[express, flask, node.js]"
2,Very efficient,5,"MongoDB, SQL, Indexing","[mongodb, sql, indexing]"
3,Very efficient,5,"SQL, MongoDB, PostgreSQL","[sql, mongodb, postgresql]"
4,"Needs improvement, delayed delivery",2,"Cypress, Manual Testing, PyTest","[cypress, manual testing, pytest]"


In [ ]:
employee_skills = (
    df.groupby('Employee_ID')['skill_list']
      .apply(lambda lists: set(sum(lists, [])))
      .to_dict()
)

agg = df.groupby('Employee_ID').agg(
    employee_avg_efficiency=('Efficiency', 'mean'),
    employee_feedback_mean=('feedback_score', 'mean'),
    tasks_done_count=('Task', 'count'),
    employee_name=('Employee Name', 'first'),
    email=('Email_ID', 'first'),
    team=('Team', 'first')
).reset_index().set_index('Employee_ID')

agg.head()


,employee_avg_efficiency,employee_feedback_mean,tasks_done_count,employee_name,email,team
Employee_ID,,,,,,
1020,79.666667,3.888889,9,Yuvraj,yuvraj2.mitblr2023@learner.manipal.edu,L1
1030,75.066667,3.533333,15,Rishit,rishit.mitblr2022@learner.manipal.edu,L1
1040,74.888889,3.555556,9,Yuvi,yuvrajk1502@gmail.com,L2
1050,79.000000,3.454545,11,Geet,geet.mitblr2022@learner.manipal.edu,L1
1060,58.800000,3.000000,10,Ritu,ritu.mitblr2022@learner.manipal.edu,L2


In [ ]:
def skill_match_ratio(task_skills, emp_skills):
    if len(task_skills) == 0:
        return 0.0
    return len(set(task_skills).intersection(emp_skills)) / len(task_skills)

rows = []
for _, r in df.iterrows():
    emp_id = r['Employee_ID']
    emp_profile = agg.loc[emp_id]

    rows.append({
        'employee_id': emp_id,
        'skill_match_ratio': skill_match_ratio(r['skill_list'], employee_skills[emp_id]),
        'employee_avg_efficiency': emp_profile['employee_avg_efficiency'],
        'employee_feedback_mean': emp_profile['employee_feedback_mean'],
        'tasks_done_count': emp_profile['tasks_done_count'],
        'team': emp_profile['team'],
        'target_efficiency': r['Efficiency']
    })

features_df = pd.DataFrame(rows)

team_encoder = LabelEncoder()
features_df['team_label'] = team_encoder.fit_transform(features_df['team'])

features_df.head()


,employee_id,skill_match_ratio,employee_avg_efficiency,employee_feedback_mean,tasks_done_count,team,target_efficiency,team_label
0,1020,1.0,79.666667,3.888889,9,L1,90,0
1,1020,1.0,79.666667,3.888889,9,L1,89,0
2,1020,1.0,79.666667,3.888889,9,L1,95,0
3,1020,1.0,79.666667,3.888889,9,L1,80,0
4,1020,1.0,79.666667,3.888889,9,L1,61,0


In [ ]:
X = features_df[['skill_match_ratio', 
                 'employee_avg_efficiency', 
                 'employee_feedback_mean', 
                 'tasks_done_count', 
                 'team_label']]
y = features_df['target_efficiency']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=200, random_state=42)
xgb_model = xgb.XGBRegressor(
    n_estimators=200, random_state=42, 
    verbosity=0, objective='reg:squarederror'
)

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

print(" Models trained successfully")


✅ Models trained successfully


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def eval_model(model, X, y, name):
    preds = model.predict(X)
    mse = mean_squared_error(y, preds)     
    rmse = np.sqrt(mse)                    
    mae = mean_absolute_error(y, preds)
    r2 = r2_score(y, preds)
    print(f"{name} -> RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.2f}")

print("🔹 Training Performance")
eval_model(lr, X_train, y_train, "Linear Regression")
eval_model(rf, X_train, y_train, "Random Forest")
eval_model(xgb_model, X_train, y_train, "XGBoost")

print("\n🔹 Test Performance")
eval_model(lr, X_test, y_test, "Linear Regression")
eval_model(rf, X_test, y_test, "Random Forest")
eval_model(xgb_model, X_test, y_test, "XGBoost")


🔹 Training Performance
Linear Regression -> RMSE: 17.13, MAE: 14.55, R2: 0.06
Random Forest -> RMSE: 16.94, MAE: 14.35, R2: 0.08
XGBoost -> RMSE: 16.91, MAE: 14.26, R2: 0.08

🔹 Test Performance
Linear Regression -> RMSE: 16.18, MAE: 13.66, R2: 0.04
Random Forest -> RMSE: 17.11, MAE: 14.40, R2: -0.07
XGBoost -> RMSE: 17.18, MAE: 14.43, R2: -0.08


In [ ]:
employee_profiles = agg.copy()
employee_profiles['skills_set'] = employee_profiles.index.map(lambda eid: employee_skills[eid])

def recommend_employees(new_task_skill_text, top_n=5, models=None):
    if models is None:
        models = [lr, rf, xgb_model]
    
    new_task_skills = normalize_skillset(new_task_skill_text)
    
    rows = []
    for eid, prof in employee_profiles.iterrows():
        X_row = pd.DataFrame([{
            'skill_match_ratio': skill_match_ratio(new_task_skills, prof['skills_set']),
            'employee_avg_efficiency': prof['employee_avg_efficiency'],
            'employee_feedback_mean': prof['employee_feedback_mean'],
            'tasks_done_count': prof['tasks_done_count'],
            'team_label': team_encoder.transform([prof['team']])[0] 
                          if prof['team'] in team_encoder.classes_ else 0
        }])
        
        preds = [m.predict(X_row)[0] for m in models]
        avg_score = float(np.mean(preds))
        
        rows.append({
            'Employee_ID': eid,
            'Employee_Name': prof['employee_name'],
            'Email': prof['email'],
            'Predicted_Efficiency': round(avg_score, 2),
            'Skill_Match': round(X_row['skill_match_ratio'].iloc[0], 2)
        })
    
    return pd.DataFrame(rows).sort_values('Predicted_Efficiency', ascending=False).head(top_n)

recommend_employees("Python, Flask, API", top_n=5)


,Employee_ID,Employee_Name,Email,Predicted_Efficiency,Skill_Match
85,1870,Fernando,employee80@company.com,87.35,0.67
16,1180,Siddharth,employee10@company.com,84.98,0.67
12,1140,Ishaan,employee5@company.com,84.10,0.33
3,1050,Geet,geet.mitblr2022@learner.manipal.edu,83.61,0.67
10,1120,Aditya,employee3@company.com,82.50,0.67


In [ ]:
recommendations_df = recommend_employees("Python, Flask, API", top_n=10)

recommendations_df['Team'] = recommendations_df['Employee_ID'].map(lambda eid: agg.loc[eid]['team'])
recommendations_df['Skills'] = recommendations_df['Employee_ID'].map(lambda eid: list(employee_skills[eid]))

recommendations_df.to_json('../models/recommendations.json', orient='records')